<a href="https://colab.research.google.com/github/younesabdolmalaky/Pistachio/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
import os
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
from keras import metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib
import random
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers, losses, activations, models

os.path.expanduser('~/.keras/models')

'/root/.keras/models'

In [ ]:
img_size = 512

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False,
                            input_shape=(600,600,3))

# freeze extraction layers
base_model.trainable = False

# add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(4096,activation="relu")(x)
x = Dense(4096,activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(2096,activation="relu")(x)
predictions = Dense(2, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# confirm unfrozen layers
for layer in model.layers:
    if layer.trainable==True:
        print(layer)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 600, 600, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 600, 600, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 600, 600, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 300, 300, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 300, 300, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 300, 300, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 150, 150, 128)     0     

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=1),
                ModelCheckpoint('model.hdf5',
                                 save_best_only=True)]

In [ ]:
opt = Adam(learning_rate=0.005)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy', metrics.Precision(), metrics.Recall()]
)

In [ ]:
image_generator = ImageDataGenerator(rescale = 1.0/255.0,
                                   rotation_range = 0.5,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest'
                                  )

image_generator_test = ImageDataGenerator(rescale = 1.0/255.0)


In [ ]:
train_data = image_generator.flow_from_directory(
    '/content/drive/MyDrive/Pistachio/train',
    target_size=(512, 512),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=32
)

Found 1717 images belonging to 2 classes.


In [ ]:
test_data = image_generator_test.flow_from_directory(
    '/content/drive/MyDrive/Pistachio/test',
    target_size=(512, 512),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=32
)

Found 431 images belonging to 2 classes.


In [ ]:
history=model.fit(train_data,
                  epochs=50,
                  validation_data=test_data,
                  validation_steps=len(test_data),
                  verbose=1,
                  callbacks=callbacks)

Epoch 1/50
54/54 [==============================] - 343s 6s/step - loss: 4.4328 - accuracy: 0.5603 - precision: 0.5579 - recall: 0.5556 - val_loss: 0.6879 - val_accuracy: 0.5731 - val_precision: 0.5731 - val_recall: 0.5731
Epoch 2/50
54/54 [==============================] - 132s 2s/step - loss: 0.6833 - accuracy: 0.5772 - precision: 0.5741 - recall: 0.5999 - val_loss: 0.6590 - val_accuracy: 0.5800 - val_precision: 0.5810 - val_recall: 0.5824
Epoch 3/50
54/54 [==============================] - 132s 2s/step - loss: 0.5911 - accuracy: 0.6814 - precision: 0.6882 - recall: 0.6686 - val_loss: 0.4782 - val_accuracy: 0.7726 - val_precision: 0.7883 - val_recall: 0.7517
Epoch 4/50
54/54 [==============================] - 132s 2s/step - loss: 0.5296 - accuracy: 0.7472 - precision: 0.7554 - recall: 0.7373 - val_loss: 0.4615 - val_accuracy: 0.8561 - val_precision: 0.8582 - val_recall: 0.8422
Epoch 5/50
54/54 [==============================] - 131s 2s/step - loss: 0.5149 - accuracy: 0.7531 - precisi